In [2]:
from keras.layers import LSTM, Dense, Input, Dropout
from keras.models import Sequential, load_model
from sklearn.preprocessing import MinMaxScaler
import yfinance as yf
import numpy as np
import pandas as pd
from dataclasses import dataclass, asdict
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [3]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [4]:
@dataclass
class modelConfig:
    ticker: str = 'NVDA'
    time_step: int = 5
    epochs: int = 20
    optimizer: str = 'adam'
    batch_size: int = 15
    learning_rate: float = 0.05
    nn_activation: str = 'relu'
    nn_max_units: int = 100
    nn_layers: int = 2
    nn_return_sequences: bool = True
    loss: str = 'mean_squared_error'
    dropout: bool = True
    dropout_value: float = 0.2
    period: str = '3mo'

    def get_parameters(self) -> dict:
        return asdict(self)

In [5]:
def create_dataset(scaled_data) -> np:
    X, y = [], []
    for i in range(len(scaled_data) - 5 - 1):
        X.append(scaled_data[i:(i + 5), 0])
        y.append(scaled_data[i + 5, 0])
    return np.array(X), np.array(y)

In [6]:
def read_yfinance(ticker: str, period: str) -> pd.DataFrame:
    nvda: pd.DataFrame = yf.Ticker(ticker).history(period)
    return pd.DataFrame(nvda['Close'])

In [7]:
ticker = 'NVDA'
period = '3mo'

In [8]:
data : pd.DataFrame = read_yfinance(ticker, period)

In [86]:
data

,Close
Date,
2024-08-13 00:00:00-04:00,116.130066
2024-08-14 00:00:00-04:00,118.069901
2024-08-15 00:00:00-04:00,122.849495
2024-08-16 00:00:00-04:00,124.569344
2024-08-19 00:00:00-04:00,129.988876
...,...
2024-11-06 00:00:00-05:00,145.610001
2024-11-07 00:00:00-05:00,148.880005
2024-11-08 00:00:00-05:00,147.630005


In [9]:
X, y = [], []

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)
X, y = create_dataset(scaled_data)
X = X.reshape(X.shape[0], X.shape[1], 1)

In [10]:
model = Sequential()
model.add(Input(shape=(X.shape[1], 1)))
model.add(LSTM(units=50, activation='relu', return_sequences=True))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [79]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

In [12]:
config = modelConfig()

In [13]:
config.get_parameters

<bound method modelConfig.get_parameters of modelConfig(ticker='NVDA', time_step=5, epochs=20, optimizer='adam', batch_size=15, learning_rate=0.05, nn_activation='relu', nn_max_units=100, nn_layers=2, nn_return_sequences=True, loss='mean_squared_error', dropout=True, dropout_value=0.2, period='3mo')>

In [14]:
config.model_evaluated = 0.34009817242622375

In [15]:
config

modelConfig(ticker='NVDA', time_step=5, epochs=20, optimizer='adam', batch_size=15, learning_rate=0.05, nn_activation='relu', nn_max_units=100, nn_layers=2, nn_return_sequences=True, loss='mean_squared_error', dropout=True, dropout_value=0.2, period='3mo')

In [80]:
model.fit(X_train, y_train, epochs=5, batch_size=10)

Epoch 1/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1591
Epoch 2/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1405 
Epoch 3/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1399 
Epoch 4/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1183
Epoch 5/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1111   


In [81]:
model.evaluate(X_test, y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.2435


0.24351376295089722

In [82]:
y_pred = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


In [63]:
y_pred

array([[[0.06841457],
        [0.11318238],
        [0.15599637],
        [0.19389458],
        [0.22858934]],

       [[0.0714587 ],
        [0.11665517],
        [0.15765218],
        [0.19573268],
        [0.2308474 ]],

       [[0.0713955 ],
        [0.11446391],
        [0.15548678],
        [0.19404878],
        [0.22864857]],

       [[0.06927532],
        [0.11226465],
        [0.1537194 ],
        [0.19179787],
        [0.22690535]],

       [[0.06972218],
        [0.11339098],
        [0.15447511],
        [0.19295664],
        [0.22691163]],

       [[0.07031652],
        [0.11361416],
        [0.1550598 ],
        [0.19246529],
        [0.22210343]],

       [[0.06978003],
        [0.11331655],
        [0.15373337],
        [0.18707302],
        [0.21773261]],

       [[0.07016397],
        [0.11282672],
        [0.14945279],
        [0.18364729],
        [0.2145221 ]],

       [[0.06915969],
        [0.10824686],
        [0.14554009],
        [0.17994386],
        [0.21333

In [83]:
y_pred.shape

(12, 5, 1)

In [52]:
y_pred = y_pred.reshape(y_pred.shape[0], -1)
y_pred.shape

(12, 5)

In [51]:
y_test

array([0.84063824, 0.81849287, 0.83434208, 0.79287327, 0.65001237,
       0.70733039, 0.72144298, 0.80524892, 0.9290037 , 1.        ,
       0.97286078, 0.92140459])

In [77]:
y_test = y_test.reshape(y_test.shape[0],1)

In [84]:
y_test

array([0.84063824, 0.81849287, 0.83434208, 0.79287327, 0.65001237,
       0.70733039, 0.72144298, 0.80524892, 0.9290037 , 1.        ,
       0.97286078, 0.92140459])

In [76]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")

ValueError: Found array with dim 3. None expected <= 2.